<a href="https://colab.research.google.com/github/Yackdu60/deGastinesRaphaelle_rendu_SIG/blob/main/deGastinesRaphaelle_rendu_SIG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Carte interactive du parc régional annuel de production d'électricité
Raphaëlle de Gastines



La production d'énergie est un enjeu clé du monde de demain. Cette carte interactive permet de visualiser la production d'électricité annuelle pour chaque région. Ce moyen de comparaison permet ainsi de connaître les mix energétiques de chaque région et leur efficacité dans le temps.

##Import et conditionnement des données

In [ ]:
from google.colab import files

In [ ]:
!pip install ipywidgets folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.8 MB/s eta 0:00:00


In [ ]:
#import ipywidgets folium
import geopandas as gpd
import folium

Téléchargement du parc annuel de production par filière (MW) au 31 décembre 2024, depuis 2008.

Les données mises à disposition prennent en compte l’ensemble des installations de production d’électricité raccordées aux réseaux publics de transport (RTE) et de distribution (Enedis, EDF SEI et les ELD).

Suite à un recalcul de l'autoconsommation photovoltaïque, le parc photovoltaïque sur les années 2022 à 2024 a augmenté.

In [ ]:
uploaded = files.upload()

Saving parc-region-annuel-production-filiere.zip to parc-region-annuel-production-filiere.zip


In [ ]:
gdf = gpd.read_file("parc-region-annuel-production-filiere.zip")

In [ ]:
gdf.info()
gdf.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   annee        221 non-null    object  
 1   code_insee_  221 non-null    object  
 2   region       221 non-null    object  
 3   parc_nuclea  119 non-null    float64 
 4   parc_thermi  221 non-null    float64 
 5   parc_hydrau  221 non-null    float64 
 6   parc_eolien  219 non-null    float64 
 7   parc_solair  221 non-null    float64 
 8   parc_bioene  220 non-null    float64 
 9   geometry     221 non-null    geometry
dtypes: float64(6), geometry(1), object(3)
memory usage: 17.4+ KB


,annee,code_insee_,region,parc_nuclea,parc_thermi,parc_hydrau,parc_eolien,parc_solair,parc_bioene,geometry
0,2024,27,Bourgogne-Franche-Comté,NaN,379.0,519.0,1179.0,1221.0,88.0,"POLYGON ((7.13026 47.50295, 7.11253 47.49511, ..."
1,2024,32,Hauts-de-France,5460.0,2404.0,4.0,6332.0,780.0,191.0,"POLYGON ((1.70435 49.23243, 1.71155 49.26448, ..."
2,2024,52,Pays de la Loire,NaN,2002.0,10.0,1897.0,1644.0,90.0,"MULTIPOLYGON (((-2.33743 46.7191, -2.31817 46...."
3,2024,53,Bretagne,NaN,1431.0,275.0,1914.0,793.0,76.0,"MULTIPOLYGON (((-3.42742 47.64103, -3.41803 47..."
4,2024,24,Centre-Val de Loire,11630.0,209.0,93.0,1727.0,1335.0,91.0,"POLYGON ((2.93662 48.16346, 2.9637 48.15308, 3..."


projection du géodataframe en lambert 93 (EPSG 2154)

In [ ]:
print(gdf.crs)

EPSG:4326


In [ ]:
gdf = gdf.to_crs(2154)

valeurs manquantes remplacées par 0

In [ ]:
gdf.fillna(0, inplace=True)

ajout du champ "somme" correspondant à la production totale d'électricité (somme de tous les parcs)

In [ ]:
gdf['somme'] = gdf['parc_nuclea']+gdf['parc_thermi']+gdf['parc_hydrau']+gdf['parc_eolien']+gdf['parc_solair']+gdf['parc_bioene']
gdf.head()


,annee,code_insee_,region,parc_nuclea,parc_thermi,parc_hydrau,parc_eolien,parc_solair,parc_bioene,geometry,somme
0,2024,27,Bourgogne-Franche-Comté,0.0,379.0,519.0,1179.0,1221.0,88.0,"POLYGON ((1010779.8 6719527.8, 1009492.1 67185...",3386.0
1,2024,32,Hauts-de-France,5460.0,2404.0,4.0,6332.0,780.0,191.0,"POLYGON ((605623.4 6904412.9, 606206.2 6907968...",15171.0
2,2024,52,Pays de la Loire,0.0,2002.0,10.0,1897.0,1644.0,90.0,"MULTIPOLYGON (((292628.7 6638105.6, 294090.5 6...",5643.0
3,2024,53,Bretagne,0.0,1431.0,275.0,1914.0,793.0,76.0,"MULTIPOLYGON (((217932.2 6746369.4, 218495.8 6...",4489.0
4,2024,24,Centre-Val de Loire,11630.0,209.0,93.0,1727.0,1335.0,91.0,"POLYGON ((695288 6784790.1, 697300.4 6783635.3...",15085.0


##Visualisation des données sous forme de carte interactive
###Parc régional annuel de production d'électricité

In [ ]:
import branca.colormap as cm

map = folium.Map(location=[46.603354, 1.888334], zoom_start=6)#zoom sur les données

global_min_somme = gdf['somme'].min()
global_max_somme = gdf['somme'].max()

colormap = cm.linear.viridis.scale(vmin=global_min_somme, vmax=global_max_somme)#création d'une échelle de couleur commune
colormap.caption = "Production totale d'électricité régionale en MW (2008-2024)"

unique_years = sorted(gdf['annee'].unique())

for year in unique_years:
    gdf_year = gdf[gdf['annee'] == year]#création des couches pour chaque année
    gdf_year.explore(#style de la carte
        m=map,
        column="somme",
        cmap=colormap,
        tooltip=["region", "parc_nuclea", "parc_thermi", "parc_hydrau", "parc_eolien", "parc_solair", "parc_bioene", "somme"],
        tooltip_kwds=dict(aliases=["Région", "parc nucléaire (MW)", "parc thermique (MW)", "parc hydraulique (MW)", "parc éolien (MW)", "parc solaire (MW)", "parc bioénergie (MW)", "production totale (MW)"]),
        popup=True,
        style_kwds={"fillOpacity": 0.7, "weight": 1},
        legend=False,
        name=f"Année {year}"
    )

folium.LayerControl().add_to(map)#ajout d'un outil de contrôle des couches
colormap.add_to(map)#ajout de l'echelle de couleur
map

In [ ]:
map.save("deGastines_Raphaelle_carte_inter.html")   #Le fichier est sauvegardé dans votre environnement Colab à gauche (icone dossier)
files.download("deGastines_Raphaelle_carte_inter.html") # Téléchargez le fichier sur votre pc

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>